# Percolación Ejercicio 1
***
## Grupo:
1. Adrián Bedón
1. Merlo José Miguel
1. Moreta Andrés
1. Ocaña Dennis
1. Ramos Xavier
***

## Qué es la percolación?
Percolación hace referencia al paso de fluidos a travez de materiales porosos. Un ejemplo de percolación es la filtracion<br>
<br>
<div>
<img src="https://cdn0.ecologiaverde.com/es/posts/3/2/1/como_hacer_un_filtro_de_agua_casero_para_beber_1123_orig.jpg" width="500"/>
</div>

***

## Simulación de Montecarlo para estudiar un fenómeno natural conocido como percolación

Para el proposito de estas simulaciones se definirá un sistema de `n * n` a modo de cuaadrícula. Dentro de esta cada cuadrante puede estar bloqueado o disponible y los cuadrantes disponibles son inicializados _vacíos_. Un sitio _lleno_ es un sitio disponible que se conecta a un sitio aledaño que también está _diponible_ siendo estos: arriba, abajo, izquierda y/o derecha).<br>
<br>
El sistema se dirá que percola si es que hay un sitio disponible _completo_ esto significando que esta conectado de un extremo a otro de la cuadrícula.<br>
<br>
<div>
<img src="https://introcs.cs.princeton.edu/python/24percolation/images/percolates-yes.png" width="300"/>
<img src="https://introcs.cs.princeton.edu/python/24percolation/images/percolates-no.png" width="300"/>
</div>

***

## Percolación vertical
Dada una matriz que representa los sitios disponibles, debemos determinar si este sistema percola. Empezando com percolación vertical siendo este el más simple ya que solo buscará percolación directamente en el prano vertical
<br>
<br>
<div>
<img src="https://introcs.cs.princeton.edu/python/24percolation/images/percolates-vertically-yes.png" width="300"/>
<img src="https://introcs.cs.princeton.edu/python/24percolation/images/percolates-vertically-no.png" width="300"/>
</div>

***

In [7]:
import numpy as np

In [146]:
def _flow(isOpen, isFull, i, j):
    n = len(isFull)
    if (i < 0) or (i >= n):
        return
    if (j < 0) or (j >= n):
        return
    if (isOpen[i][j] == 0):
        return
    if (isFull[i][j] == 1):
        return
    isFull[i][j] = 1
    _flow(isOpen, isFull, i+1, j  )  # Down.
    _flow(isOpen, isFull, i  , j+1)  # Right.
    _flow(isOpen, isFull, i  , j-1)  # Left.
    _flow(isOpen, isFull, i-1, j  )  # Up.

In [143]:
def flow(isOpen):
    n = len(isOpen)
    isFull = np.zeros((n,n),dtype=int)
    for j in range(n):
        _flow(isOpen, isFull, 0, j)
    print(isFull)
    return isFull

In [138]:
def percolates(isOpen):
    
    # Compute the full sites of the system.
    isFull = flow(isOpen)
    
    
    # If any site in the bottom row is full, then the system
    # percolates.
    n = len(isFull)
    for j in range(n):
        if (isFull[n-1][j]):
            return True
    return False


In [112]:
def armarArray(n, probabilidadDisponible):
    array = np.random.binomial(1, probabilidadDisponible, size=(n,n))
    return array

In [113]:
## recibe n y m que son valores para las dimensiones de la matriz
def main(n, probabilidadDisponible):
    array = armarArray(n, probabilidadDisponible)
    print(array)
    print(percolates(array))
    
        

In [156]:
## Ejecuta el metodo main que toma las dimensiones n*n de la matriz 
## y un numero float entre 0 y 1 el cual representara la 
## probabilidad de asumir 1 o disponible
main(5, 0.5)

[[0 1 1 0 0]
 [1 0 1 1 1]
 [0 0 1 0 0]
 [0 0 1 1 0]
 [0 1 1 0 1]]
[[0 1 1 0 0]
 [0 0 1 1 1]
 [0 0 1 0 0]
 [0 0 1 1 0]
 [0 1 1 0 0]]
True
